# Ordinary differential equation model with probabilistic integration using ProbNumDiffEq.jl
Simon Frost (@sdwfrost), 2022-02-23

## Introduction

The classical ODE version of the SIR model is:

- Deterministic
- Continuous in time
- Continuous in state

Integration of an ODE is subject to error; one way to capture this error is by probabilistic integration. This tutorial shows how to apply probabilistic integration to an ODE model using solvers from the [ProbNumDiffEq.jl](https://github.com/nathanaelbosch/ProbNumDiffEq.jl) package.

## Libraries

In [ ]:
using ProbNumDiffEq
using Random
using Statistics
using Plots
using BenchmarkTools

## Transitions

The following function provides the derivatives of the model, which it changes in-place. State variables and parameters are unpacked from `u` and `p`; this incurs a slight performance hit, but makes the equations much easier to read.

In [ ]:
function sir_ode!(du,u,p,t)
    (S,I,R) = u
    (β,c,γ) = p
    N = S+I+R
    @inbounds begin
        du[1] = -β*c*I/N*S
        du[2] = β*c*I/N*S - γ*I
        du[3] = γ*I
    end
    nothing
end;

## Time domain

We set the timespan for simulations, `tspan`, initial conditions, `u0`, and parameter values, `p`.

In [ ]:
δt = 0.1
tmax = 40.0
tspan = (0.0,tmax);

## Initial conditions

In [ ]:
u0 = [990.0,10.0,0.0]; # S, I, R

## Parameter values

In [ ]:
p = [0.05,10.0,0.25]; # β, c, γ

## Random number seed

In [ ]:
Random.seed!(1234);

## Running the model

In [ ]:
prob = ODEProblem(sir_ode!, u0, tspan, p);

To use probabilistic integration, we just use one of the solvers from ProbNumDiffEq.jl. We'll use the `EK0` and the `EK1` solvers to compare their output. More information on the solvers can be found [here](https://nathanaelbosch.github.io/ProbNumDiffEq.jl/dev/solvers/).

In [ ]:
sol_ek0 = solve(prob,
                EK0(prior=IWP(3), order=3, diffusionmodel=DynamicDiffusion(), smooth=true),
                dt=δt,
                abstol=1e-1,
                reltol=1e-2);

In [ ]:
sol_ek1 = solve(prob,
                EK1(prior=IWP(3), order=3, diffusionmodel=DynamicDiffusion(), smooth=true),
                dt=δt,
                abstol=1e-1,
                reltol=1e-2);

## Post-processing

We can look at the mean and standard deviation by examining the `pu` field of the solution. The following gives the mean and covariance of `S`, `I`, and `R` at `t=20.0` for the two solvers.

In [ ]:
s20_ek0 = sol_ek0(20.0)
mean(s20_ek0),cov(s20_ek0)

In [ ]:
s20_ek1 = sol_ek1(20.0)
mean(s20_ek1),cov(s20_ek1)

We can also take samples from the trajectory using `ProbNumDiffEq.sample`.

In [ ]:
num_samples = 100
samples_ek0 = ProbNumDiffEq.sample(sol_ek0, num_samples);
samples_ek1 = ProbNumDiffEq.sample(sol_ek1, num_samples);

## Plotting

We can now plot the results; there is a default plotting method (e.g. using `plot(sol_ek1)`), but the below accentuates the differences between samples (although it is low in this case, even on a log scale).

In [ ]:
p_ek0 = plot(sol_ek0.t,
         samples_ek0[:, :, 1],
         label=["S" "I" "R"],
         color=[:blue :red :green],
         xlabel="Time",
         ylabel="Number",
         title="EK0")
for i in 2:num_samples
    plot!(p_ek0,
          sol_ek0.t,
          samples_ek0[:, :, i],
          label="",
          color=[:blue :red :green])
end;

In [ ]:
p_ek1 = plot(sol_ek1.t,
         samples_ek1[:, :, 1],
         label=["S" "I" "R"],
         color=[:blue :green],
         xlabel="Time",
         ylabel="Number",
         title="EK1")
for i in 2:num_samples
    plot!(p_ek1,
          sol_ek1.t,
          samples_ek1[:, :, i],
          label="",
          color=[:blue :red :green],)
end;

This shows the simulations around the peak.

In [ ]:
plot(p_ek0, p_ek1, layout = (1,2), xlim=(15,20), ylim=(100,1000), yaxis=:log10)

This shows the simulations around the end of the timespan.

In [ ]:
plot(p_ek0, p_ek1, layout = (1,2), xlim=(35,40), ylim=(10,1000), yaxis=:log10)

## Benchmarking

In [ ]:
@benchmark solve(prob,
                 EK0(prior=IWP(3), order=3, diffusionmodel=DynamicDiffusion(), smooth=true),
                 abstol=1e-1,
                 reltol=1e-2)

In [ ]:
@benchmark solve(prob,
                 EK1(prior=IWP(3), order=3, diffusionmodel=DynamicDiffusion(), smooth=true),
                 abstol=1e-1,
                 reltol=1e-2)